In [ ]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import eurocodepy as ec


In [ ]:
code = 'CEN-2'
coef_imp = 'iii'
soil = 'C'
zone = '.4g'
behaviour = 2.0
txt = code + '_' + coef_imp + '_' + soil + '_' + zone

S, a_g, TB, TC, TD = ec.ec8.get_spectrum_parameters(code, coef_imp, soil, zone)
S = 1.5
a_g = 3.2

ec.ec8.draw_spectrum_ec8(ec.ec8.spectrum_user(a_g, S, behaviour, TB, TC, TD, 0.2))

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact, interact_manual

behaviour = widgets.FloatSlider(
    value=2.0,min=1.5,max=5.0,step=0.1,description='behaviour factor:',readout_format='.1f'
)
code = widgets.Dropdown(
    options=[(
        'Portugal Type 1', 'PT-1'), ('Portugal Type 2 (continent)', 'PT-2'), ('Portugal Type 2 (Azores)', 
        'PT-A'), ('CEN standard Type 1', 'CEN-1'), ('CEN standard Type 2', 'CEN-2')],
    value='PT-1',
    description='Spectrum:',
)
soil = widgets.Dropdown(
    options=[('Type A', 'A'), ('Type B', 'B'), ('Type C', 'C'), ('Type D', 'D'), ('Type E', 'E')],
    value='A',
    description='Soil:',
)
imp_coef = widgets.Dropdown(
    options=[('Class I (CC1)', 'i'), ('Class II (CC2)', 'ii'), ('Class III (CC3)', 'iii'), ('Class IV (CC4)', 'iv')],
    value='ii',
    description='Importance:',
)
zone = widgets.Dropdown(
    options=[('Space " "', ' '), ('Comma ","', ','), ('Semicolon ";"', ';')],
    value=' ',
    description='Zone:',
)
zone = widgets.Dropdown(
    options=['1.1', '1,2', '1.3', '1.4', '1.5', '1.6'],
    value='1.1',
    description='Zone:',
)
separator = widgets.Dropdown(
    options=[('Space " "', ' '), ('Comma ","', ','), ('Semicolon ";"', ';')],
    value=' ',
    description='Separator:',
)
button_write = widgets.Button(
    description='Write spectrum',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Writes a spectrum .txt file for SAP2000',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button_draw = widgets.Button(
    description='Draw spectrum',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Draw the spectrum',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)


def handle_slider_change(change):
    if code.value == 'PT-2':  
        zone.options = ['2.1', '2.2', '2.3', '2.4', '2.5']
    elif code.value == 'PT-A':
        zone.options = ['2.1', '2.2', '2.3', '2.4', '2.5']
    elif code.value == 'PT-1':
        zone.options = ['1.1', '1.2', '1.3', '1.4', '1.5', '1.6']
    else:
        zone.options = ['.1g', '.2g', '.3g', '.4g', '.5g', '.6g', '.7g', '.8g', '.9g', '1.0g']

code.observe(handle_slider_change, names='value')


def on_write_clicked(b):
    spec = ec.ec8.spectrum_ec8(code.value, imp_coef.value, soil.value, zone.value, behaviour.value)
    ec.ec8.write_spectrum_ec8(spec, separator.value)


def on_draw_clicked(b):
    spec = ec.ec8.spectrum_ec8(code.value, imp_coef.value, soil.value, zone.value, behaviour.value)
    ec.ec8.draw_spectrum_ec8(spec)


button_draw.on_click(on_draw_clicked)
button_write.on_click(on_write_clicked)

tab1 = VBox([code, soil, imp_coef, zone, behaviour])
tab2 = VBox([HBox([button_write, separator]), button_draw])
tab = widgets.Tab()
tab.children = [tab1, tab2]
tab.set_title(0, 'Data')
tab.set_title(1, 'Operations')
tab
#display(VBox([code, soil, imp_coef, zone, separator, button_write, button_draw]))